# Imports


Standard Library & Package Imports


In [ ]:
import itertools
from functools import partial
from typing import List

import torch

Project Imports


In [ ]:
from attacks.weight_attack import weight_attack
from federated.client import (
    AdversarialClient,
    PrivateClient,
    PrivateAdversarialClient,
    PublicClient,
    TClient
)
from federated.server import Server
from data_loaders.mnist.data_loader import DataLoader
from models.mnist.mnist_cnn import MnistCNN as Model
from setup import FederatedLearningConfig 
from training import train_model, test_model 
from utilities import parse_filename_to_config, save_results

# Setting up the config, dataloader, clients, and server


In [ ]:
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
elif torch.cuda.is_available():
    DEVICE = torch.device("cuda") 
else:
    DEVICE = torch.device("cpu")

In [ ]:
def calculate_trustworthy_thresholds(n_clients):
    return [
        0,
        1 / (1.1 * n_clients),
        1 / (1.3 * n_clients),
        1 / (1.5 * n_clients),
        1 / (2.0 * n_clients),
    ]

Get the configuration, torch device, and data loader


In [ ]:
CONFIG_OPTIONS = {
    "n_clients": [1, 2, 5, 10],
    "n_adv": [0, 1, 2, 5],
    "noise_multiplier": [0.001, 0.01, 0.1, 1, 10],
    "n_rounds": [20],
    "L": [-1],
    "batch_size": [64],
    "should_use_iid_training_data": [False, True],
    "should_enable_adv_protection": [False, True],
    "should_use_private_clients": [False],
    "target_epsilon": [None],
    "target_delta": [None],
}


# Function to create configurations with dynamic trustworthy thresholds
def create_configs(configs):
    # Get the keys and values, excluding trustworthy_threshold
    keys = list(configs.keys())
    values = list(configs.values())
    
    # Generate initial configurations without trustworthy thresholds
    initial_configs = [dict(zip(keys, v)) for v in itertools.product(*values)]
    
    # Extend configurations with dynamic trustworthy thresholds
    extended_configs = []
    for config in initial_configs:
        n_clients = config['n_clients']
        thresholds = calculate_trustworthy_thresholds(n_clients)
        # always include 0 threshold
        config['trustworthy_threshold'] = 0
        extended_configs.append(config.copy())

        if config["should_enable_adv_protection"]:
            for threshold in thresholds[1:]:
                extended_config = config.copy()
                extended_config['trustworthy_threshold'] = threshold
                extended_configs.append(extended_config)

    # filter out configs where n_adv > n_clients
    extended_configs = [config for config in extended_configs if config['n_adv'] < config['n_clients']]
    # filter out configs where noise_multiplier != 0.001 and n_adv == 0
    extended_configs = [config for config in extended_configs if config['noise_multiplier'] == 0.001 or config['n_adv'] != 0]
    # filter out configs where should_enable_adv_protection == True and n_adv == 0
    extended_configs = [config for config in extended_configs if not config['should_enable_adv_protection'] or config['n_adv'] != 0]
    # filter out all n_adv > 0 configs when n_clients == 1
    extended_configs = [config for config in extended_configs if config['n_clients'] > 1 or config['n_adv'] == 0]
    # filter out should_enable_adv_protection == True configs when n_clients == 1
    extended_configs = [config for config in extended_configs if config['n_clients'] > 1 or not config['should_enable_adv_protection']]
    # filter out n_clients = 1 and should_use_iid_training_data = False
    extended_configs = [config for config in extended_configs if config['n_clients'] > 1 or config['should_use_iid_training_data']]

    extended_configs = [FederatedLearningConfig(**config) for config in extended_configs]
    
    return extended_configs

configs = create_configs(CONFIG_OPTIONS)

In [ ]:
# get completed configs from the results/ directory
import os
directory_files = os.listdir('results/')
server_files = [file for file in directory_files if '_server' in file]

completed_configs = [
    parse_filename_to_config(file)
    for file in server_files
]


In [ ]:
configs_to_run = [config for config in configs if config not in completed_configs]

In [ ]:
configs_to_run, len(configs_to_run)

# Training Loop


In [ ]:
for c_i, cfg in enumerate(configs_to_run):
    print(f"Running configuration {c_i + 1}/{len(configs_to_run)}")
    print(cfg)

    batch_size = cfg.batch_size
    enable_adv_protection = cfg.should_enable_adv_protection
    n_adv = cfg.n_adv
    n_clients = cfg.n_clients
    noise_multiplier = cfg.noise_multiplier
    num_rounds = cfg.n_rounds
    should_use_private = cfg.should_use_private_clients
    target_delta = cfg.target_delta
    target_epsilon = cfg.target_epsilon
    trust_threshold = cfg.trustworthy_threshold
    use_iid = cfg.should_use_iid_training_data

    attack = partial(weight_attack,
                     noise_multiplier=noise_multiplier)

    dataloader = DataLoader(
            batch_size=batch_size,
            device=DEVICE,
            test_split=0.2,
            val_split=0.2,
            n_clients=n_clients,
            use_iid=use_iid,
        )

    adv_clients: List[TClient] = [
            AdversarialClient(
                id=f"Adversarial Client {i}",
                model=Model(),
                device=DEVICE,
                data_loader=dataloader.train_loaders[i],
                attack=attack,
            )
            if not should_use_private
            else PrivateAdversarialClient(
                id=f"Private Adversarial Client {i}",
                model=Model(),
                device=DEVICE,
                data_loader=dataloader.train_loaders[i],
                target_epsilon=target_epsilon,
                target_delta=target_delta,
                num_epochs=num_rounds,
                max_grad_norm=100.0,
                attack=attack,
            )
            for i in range(n_adv)
        ]
    non_adv_clients: List[TClient] = [
            (
                PublicClient(
                    id=f"Client {i}",
                    model=Model(),
                    device=DEVICE,
                    data_loader=dataloader.train_loaders[i],
                )
            )
            if not should_use_private
            else (
                PrivateClient(
                    id=f"Private Client {i}",
                    model=Model(),
                    device=DEVICE,
                    data_loader=dataloader.train_loaders[i],
                    target_epsilon=target_epsilon,
                    target_delta=target_delta,
                    num_epochs=num_rounds,
                    max_grad_norm=100.0,
                )
            )
            for i in range(n_adv, n_clients)
        ]
    clients = adv_clients + non_adv_clients

    server = Server(
        model=Model(),
        device=DEVICE,
        validation_data=dataloader.val_loader,
        enable_adversary_protection=enable_adv_protection,
        weight_threshold=trust_threshold,
    )

    train_model(
        server=server,
        num_rounds=num_rounds,
        clients=clients,
        L=cfg.L,
        is_verbose=False
    )

    test_model(
        model=server.model,
        test_loader=dataloader.test_loader,
    )

    save_results(
        server=server,
        clients=clients,
        config=cfg,
    )